# Workshop DL01 - Intro to Deep Learning

### Setup

#### Installing TensorFlow
1. Open the Anaconda3 Terminal
2. Execute `pip install --upgrade tensorflow`

#### Getting the MNIST Data
1. Go to https://www.kaggle.com/c/digit-recognizer/data
2. Download "train.csv" and "test.csv", put these into the same folder as this notebook

# Deep Neural Network Image Classifier

The following changes increased test score substantially (from ~70% to 90%+):
* Changed cost function to softmax_cross_entropy_with_logits; required transforming input labels to one_hot
* Switched learning rate to 0.001 (default for AdamOptimizer)
* Changed initialization of weights and biases by using get_variable with xavier_initializer rather than truncated_normal (for weights) and constant (for biases)

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

/Users/evan/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# read-in raw data
data = pd.read_csv("datasets/mnist_train.csv")

# mask for splitting data into train and test sets
mask = np.random.rand(len(data)) < 0.90

# training data, features (X) and labels (y)
train_X = data[mask].drop("label", axis=1)
train_y = data.label[mask]

# test data, features (X) and labels (y)
test_X = data[~mask].drop("label", axis=1)
test_y = data.label[~mask]

In [7]:
# (1) define network architecture
with tf.variable_scope("all", reuse=tf.AUTO_REUSE):
    
    with tf.name_scope("input"):
        x = tf.placeholder(tf.float32, [None, 784], name="features")
        y = tf.placeholder(tf.int64, [None], name="labels")
        y_one_hot = tf.one_hot(indices=y, depth=10)

    with tf.name_scope("fc-layer-1"):
        w1 = tf.get_variable(name="w1", shape=[784,500],
            initializer=tf.contrib.layers.xavier_initializer())
        b1 = tf.get_variable(name="b1", shape=[500])
        fc1 = tf.nn.relu(tf.matmul(x, w1) + b1)

    with tf.name_scope("fc-layer-2"):
        w2 = tf.get_variable(name="w2", shape=[500,500],
            initializer=tf.contrib.layers.xavier_initializer())
        b2 = tf.get_variable(name="b2", shape=[500])
        fc2 = tf.nn.relu(tf.matmul(fc1, w2) + b2)

    with tf.name_scope("output"):
        w3 = tf.get_variable(name="w3", shape=[500,10],
            initializer=tf.contrib.layers.xavier_initializer())
        b3 = tf.get_variable(name="b3", shape=[10])
        y_ = tf.matmul(fc2, w3) + b3

In [8]:
# (2) define loss function
with tf.name_scope("loss"):
    cross_entropy = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(
        logits=y_, labels=y_one_hot)
    )

In [9]:
# (3) define optimizer
with tf.name_scope("optimizer"):
    optimizer = tf.train.AdamOptimizer().minimize(
        cross_entropy
    )

In [10]:
# aside: evaluation operations
with tf.name_scope("accuracy"):
    correct = tf.equal(tf.argmax(y_, 1), y)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [11]:
# (4) train network
with tf.Session() as sess:
    
    # initialize all our operations
    sess.run(tf.global_variables_initializer())
    
    # train the network
    print("==================================================")
    for epoch in range(50):
        
        # Note: usually you have to chunk through the data in another
        #       for loop; the data here is small though, so you might not
        #       have to.
        _, epoch_loss = sess.run([optimizer, cross_entropy],
                feed_dict={x: train_X, y: train_y})
        
        print("Epoch " + str(epoch+1) + " completed, loss = "
              + str(epoch_loss))
        
        # evaluate on test data
        score = sess.run([accuracy],
                feed_dict={x: test_X, y: test_y})
        print("Test accuracy = " + str(score))
        
        print("==================================================")

Epoch 1 completed, loss = 85.27442
Test accuracy = [0.21798816]
Epoch 2 completed, loss = 86.67346
Test accuracy = [0.30153847]
Epoch 3 completed, loss = 89.09045
Test accuracy = [0.44260356]
Epoch 4 completed, loss = 63.186314
Test accuracy = [0.61278105]
Epoch 5 completed, loss = 39.160202
Test accuracy = [0.65964496]
Epoch 6 completed, loss = 28.862825
Test accuracy = [0.6710059]
Epoch 7 completed, loss = 20.810696
Test accuracy = [0.728284]
Epoch 8 completed, loss = 13.946636
Test accuracy = [0.76449704]
Epoch 9 completed, loss = 9.745537
Test accuracy = [0.7635503]
Epoch 10 completed, loss = 9.852828
Test accuracy = [0.7443787]
Epoch 11 completed, loss = 11.816876
Test accuracy = [0.756213]
Epoch 12 completed, loss = 10.804301
Test accuracy = [0.79952663]
Epoch 13 completed, loss = 8.19464
Test accuracy = [0.82769233]
Epoch 14 completed, loss = 6.5099053
Test accuracy = [0.8421302]
Epoch 15 completed, loss = 5.6846085
Test accuracy = [0.85893494]
Epoch 16 completed, loss = 5.13373